In [ ]:
2) Ejemplo de uso (rápido) en un notebook o script

In [ ]:
import pandas as pd
from ma_mmm_package.pipelines.run_mmm import MMMConfig, run_mmm_for_series

df = pd.read_csv("data/raw/your_dataset.csv")

cfg = MMMConfig(
    target_col="ALL_PURCHASES",
    theta=0.5,
    sat_alpha=1.0,
    ridge_alpha=1.0,
    add_trend=True,
    add_seasonality=True,
)

out = run_mmm_for_series(
    df=df,
    cfg=cfg,
    org_id="882ce7e286d66facc66518783e2192c7",
    territory_name="All Territories",
)

out["results"], out["coef_summary"].head(10)


In [ ]:
3) Cómo aplicarlo a muchas organizaciones (loop)

In [ ]:
pairs = (
    df[["ORGANISATION_ID", "TERRITORY_NAME"]]
    .drop_duplicates()
    .values
)

all_runs = []
for org_id, terr in pairs:
    try:
        out = run_mmm_for_series(df, cfg, org_id, terr)
        summary = out["results"].assign(
            ORGANISATION_ID=org_id,
            TERRITORY_NAME=terr
        )
        all_runs.append(summary)
    except Exception as e:
        # series too short / no spends / no splits etc.
        continue

all_results = pd.concat(all_runs, ignore_index=True)
all_results.groupby(["ORGANISATION_ID","TERRITORY_NAME"])["r2_test"].mean().sort_values(ascending=False).head(20)
